In [ ]:
pip install google-api-python-client

In [ ]:
###Importing neccessary modules

from googleapiclient.discovery import build
import pandas as pd
import json

In [ ]:
###Credentials for connecting to youtube for fetching data

api_key="###"
api_service_name = "##"
api_version = "##"

###channel ids of different channels of youtube
channel_id=["UCk7NcgnqCmui1AV7MTXZwOw",'UCnz-ZXXER4jOvuED5trXfEA',	'UCbTggJVf0NDTfWX-C_gUGSg','UCLXHSvxpM4U1lbTzeXycqrA','UCChmJrVa8kDg05JfCmxpLRw','UC7cs8q-gJRlGwj4A8OmCmXg','UCAq9f7jFEA7Mtl3qOZy2h1A','UCeVMnSShP_Iviwkknt83cww','UCacvJAgrPTjSEdnZObMzpqQ']

In [ ]:
youtube=build(api_service_name,api_version,developerKey=api_key)

In [ ]:
### creating list of neccesaary data

Data=[]

for i in channel_id:

    request=youtube.channels().list(part='snippet,contentDetails,statistics',id=i)
    response=request.execute()
    data=dict(Channel_Id=i,Channel_name=response['items'][0]['snippet']['title'],
              Published_at=response['items'][0]['snippet']['publishedAt'],
              Video_count=response['items'][0]['statistics']['videoCount'],
              View_count=response['items'][0]['statistics']['viewCount'] ,
              Subscriber_count=response['items'][0]['statistics']['subscriberCount'],
              Playlist_id=response['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
    Data.append(data)

In [ ]:
###converting list to dataframe

df=pd.DataFrame(Data)

In [ ]:
## changing datatypes as per requirement

df['Video_count']=df['Video_count'].astype(int)
df['View_count']=df['View_count'].astype(int)
df['Subscriber_count']=df['Subscriber_count'].astype(int)


In [ ]:
df['Published_at']=pd.to_datetime (df['Published_at'],format='ISO8601')

In [ ]:
import boto3

In [ ]:
##credentials for connecting to S3

s3 = boto3.client(
    's3',
    aws_access_key_id='#####',
    aws_secret_access_key='###',
    region_name='###'
)

In [ ]:
###Uploading the data to S3 

from io import BytesIO
import io

"""buffer = BytesIO()
df.to_parquet(buffer, index=False)
buffer.seek(0)"""
csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=False)

s3.put_object(Bucket='sp-yt-analysis', Key='input files/Channel_overview.csv', Body=csv_buffer.getvalue())


In [ ]:
playlist_id=df['Playlist_id'].tolist()

In [ ]:

### creating list of data
VID=[]
for m in playlist_id:




    req=youtube.playlistItems().list(part='contentDetails',playlistId=m,maxResults=50)
    resp=req.execute()
    for i in range(len(resp['items'])):
                  Dt=dict(Playlist_id=m,Video_id=resp['items'][i]['contentDetails']['videoId'],
                  Video_published_at=resp['items'][i]['contentDetails']['videoPublishedAt'] )
                  VID.append(Dt)
                 
    next_page_token=resp.get('nextPageToken')
    More_pages=True
    while More_pages:
         if next_page_token is None :
            More_pages=False
         else :
             req=youtube.playlistItems().list(part='contentDetails',playlistId=m,maxResults=50,pageToken=next_page_token)
             resp=req.execute()
             for i in range(len(resp['items'])):
                  Dt=dict(Playlist_id=m,Video_id=resp['items'][i]['contentDetails']['videoId'],
                  Video_published_at=resp['items'][i]['contentDetails']['videoPublishedAt'] )
                  VID.append(Dt)
             
             next_page_token=resp.get('nextPageToken')
    


In [ ]:

###converting list to dataframe
df1=pd.DataFrame(VID)

In [ ]:
## changing datatypes as per requirement
df1['Video_published_at']=pd.to_datetime (df1['Video_published_at'],format='ISO8601')

In [ ]:
"""buffer = BytesIO()
df1.to_parquet(buffer, index=False)
buffer.seek(0)"""
 ###Uploading the data to S3 
import io
csv_buffer = io.StringIO()
df1.to_csv(csv_buffer, index=False)

s3.put_object(Bucket='sp-yt-analysis', Key='input files/Playlist_overview.csv', Body=csv_buffer.getvalue())